In [1]:
import pickle
import os
import pandas as pd

In [27]:
train_pk = "/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/out_embeds_train.pickle"
test_pk = "/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/out_embeds_test.pickle"
val_pk = "/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/out_embeds_val.pickle"

In [28]:
#load pickles
with open(train_pk, 'rb') as f:
    train_data = pickle.load(f)

with open(test_pk, 'rb') as f:
    test_data = pickle.load(f)

with open(val_pk, 'rb') as f:
    val_data = pickle.load(f)
    

In [29]:
len(train_data), len(test_data), len(val_data)

(24548, 3224, 3222)

In [30]:
for patient in test_data.keys():
    train_data[patient] = test_data[patient]

In [31]:
for patient in val_data.keys():
    train_data[patient] = val_data[patient]

In [32]:
#save train data in to ct_out_embeds.pickle
with open("/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/all_ct_out_embeds.pickle", 'wb') as f:
    pickle.dump(train_data, f)

In [33]:
train_csv_path = "/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/labels_train.csv"
test_csv_path = "/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/labels_test.csv"
val_csv_path = "/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/labels_val.csv"

In [34]:
#load csvs and concatenate them
train_csv = pd.read_csv(train_csv_path)
test_csv = pd.read_csv(test_csv_path)
val_csv = pd.read_csv(val_csv_path)

train_csv = pd.concat([train_csv, test_csv, val_csv], axis=0)

#update index
train_csv.reset_index(drop=True, inplace=True)

#save updated csv
train_csv.to_csv("/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/ct_labels.csv", index=False)

In [35]:
splits_csv = "/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/Labels.csv"
splits = pd.read_csv(splits_csv)



In [36]:
splits = splits[['idx', 'label', 'pe_type', 'split']]
splits.rename(columns={'label': 'target'}, inplace=True)
splits.head()

,idx,target,pe_type,split
0,1436,0,NaN,train
1,1880,1,segmental,train
2,2738,0,NaN,val
3,2883,0,NaN,train
4,2302,1,segmental,train


In [37]:
merged_df = pd.merge(train_csv, splits, left_on='study_num', right_on='idx', how='left')
merged_df.drop('idx', axis=1, inplace=True)

merged_df.head()

,study_num,slice_idx,label,target,pe_type,split
0,1436,0,0,0,NaN,train
1,1436,24,0,0,NaN,train
2,1436,48,0,0,NaN,train
3,1436,72,0,0,NaN,train
4,1436,96,0,0,NaN,train


In [38]:
merged_df.to_csv("/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/all_ct_labels_split.csv", index=False)

In [26]:
merged = pd.read_csv("/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/ct_labels_split.csv")

merged_450 = merged[merged['study_num']==291]
#print all

       study_num  slice_idx  label  target    pe_type  split
11320        291          0      0       1  segmental  train
11321        291         24      0       1  segmental  train
11322        291         48      0       1  segmental  train
11323        291         72      1       1  segmental  train
11324        291         96      0       1  segmental  train
...          ...        ...    ...     ...        ...    ...
11424        291       2496      0       1  segmental  train
11425        291       2520      0       1  segmental  train
11426        291       2544      0       1  segmental  train
11427        291       2568      0       1  segmental  train
11428        291       2592      0       1  segmental  train

[109 rows x 6 columns]


In [39]:
import pandas as pd
from torch.utils.data import Dataset
import pickle
import torch

class UltrasonicDataset(Dataset):
    def __init__(self, pkl_file, csv_file, split, transform=None):
        with open(pkl_file, 'rb') as f:
            self.pkl = pickle.load(f)

        self.csv = pd.read_csv(csv_file)
        #sort by study_num
        self.csv = self.csv.sort_values(by=['study_num', 'slice_idx'])
        self.transform = transform
        self.split = split
        self.study_num = []
        self.data = []
        cur_id = -1
        for i in range(len(self.csv)):
            if self.csv['split'][i] != self.split:
                continue
            if i == 0 or self.csv['study_num'][i] != self.csv['study_num'][i-1]:
                self.study_num.append(self.csv['study_num'][i])
                cur_id += 1
                self.data.append({"embeddings": [self.pkl[f"{self.csv['study_num'][i]}_{self.csv['slice_idx'][i]}"]], "label": self.csv['target'][i]})

            else:
                self.data[cur_id]["embeddings"].append(self.pkl[f"{self.csv['study_num'][i]}_{self.csv['slice_idx'][i]}"])
        
        print(f"Loaded {len(self.data)} samples")
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        target = int(self.data[idx]["label"])
        embeddings = self.data[idx]["embeddings"]
        embeddings = torch.stack(embeddings, dim=0)

        study_num = self.study_num[idx]

        if self.transform:
            embeddings = self.transform(embeddings)
        
        return study_num, embeddings, target

In [40]:
dataset = UltrasonicDataset(pkl_file="/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/all_ct_out_embeds.pickle", csv_file="/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/all_ct_labels_split.csv", split="train")

Loaded 1454 samples


In [41]:
dataset_val = UltrasonicDataset(pkl_file="/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/all_ct_out_embeds.pickle", csv_file="/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/all_ct_labels_split.csv", split="val")

Loaded 193 samples


In [42]:
dataset_test = UltrasonicDataset(pkl_file="/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/all_ct_out_embeds.pickle", csv_file="/home/nuren.zhaksylyk/Documents/HC701/project/PEFormer/data/ct/all_ct_labels_split.csv", split="test")

Loaded 190 samples


In [30]:
merged_df['study_num'][100]

2956

In [43]:
train_csv = pd.read_csv(train_csv_path)
test_csv = pd.read_csv(test_csv_path)
val_csv = pd.read_csv(val_csv_path)

len(train_csv['study_num'].unique()), len(test_csv['study_num'].unique()), len(val_csv['study_num'].unique())

(1429, 162, 164)

In [45]:
max = 0

sum = 0

for study_num, emb, target in dataset_test:
    
    if emb.shape[0] > max:
        max = emb.shape[0]
        print(f"{study_num}:{emb.shape}")
    sum += emb.shape[0]

print(max, sum/len(dataset))

#109 maximum number of slices in a single

2506:torch.Size([20, 2048])
2832:torch.Size([21, 2048])
15:torch.Size([23, 2048])
1222:torch.Size([24, 2048])
2465:torch.Size([26, 2048])
2387:torch.Size([34, 2048])
306:torch.Size([36, 2048])
36 2.217331499312242
